In [1]:
import numpy as np 
import pandas as pd 
import os    
from copy import copy 
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
np.random.seed(42)

def crossover(parent1: np.array, parent2: np.array):  
    #定义一个交叉函数，接受两个 NumPy 数组类型的父代（parent1 和 parent2）
    """
    Crossover based on the pool of samples
    Args:
        parent1: the first parent for crossover
        parent2: the second parent for crossover
    Returns:
        offspring: the offspring after crossover
    """

    parent1_temp = copy(parent1)                     #复制父代序列，以避免直接修改原始数据
    parent2_temp = copy(parent2)                     #复制父代序列，以避免直接修改原始数据

    assert parent1_temp.shape== parent2_temp.shape   #断言检查，确保两个父代的形状相同。如果不相同，程序会抛出异常
    offspring=[]                                     #初始化一个空列表用于存储子代序列

    for i in range(parent1_temp.shape[0]):           #遍历父代序列的每个元素。假设父代序列是一维的，如果是多维的需要改动
        if np.random.randint(2) == 0:                #随机选择一个在 [0, 1] 之间的整数。如果为 0，则来自 parent1。如果为 1，则来自 parent2
            offspring.append(parent1_temp[i])        
        else:
            offspring.append(parent2_temp[i])        #根据条件，选择一个父代中的元素并添加到子代列表中
    offspring = np.array(offspring)                  #将子代列表转换为 NumPy 数组

    return offspring.reshape(parent1_temp.shape)     #返回与父代形状相同的子代数组

def mutation(offspring: np.array, mutate_rate: float= 0.4, possible_pool: list = None):
    #定义一个变异函数，接受一个子代序列（NumPy 数组）、变异率（默认 0.2）和一个可能的氨基酸池
    """
    Mutate the genotype of the offsprings
    Args:
        offspring: the offspring to be evaluated. it should be input_number*1 for continous variables, 
                or input_number*categorical_size for categorical variables.
        mutate_rate: the probability of mutation
        possible_pool: a pool contaning all possible compounds that can be selected
    Returns:
        mutated_offspring: the mutated offspring
    """
    offspring = copy(offspring)              #复制子代序列以避免直接修改原始数据

    for i in range(len(offspring)):          #遍历子代序列的每个元素
        if i == 4:                           #确保第五个氨基酸为K
            if np.random.uniform() < mutate_rate:
                new_amino_acid = 'K'
                offspring[i] = new_amino_acid
        else:
            if np.random.uniform() < mutate_rate:
                # Exclude 'K' from the possible pool for other positions
                non_k_pool = [aa for aa in possible_pool if aa != 'K']
                new_amino_acid = np.random.choice(non_k_pool, size=1).item()
                offspring[i] = new_amino_acid
            
    return offspring                         #返回变异后的子代序列

In [ ]:
data = pd.read_excel("<file path to experimental data>")

In [3]:
X = np.array([i.replace("-重复", "").split("-")[1].split(",") for i in data["exp"]])  #把重复数据的“重复”删除，拆分为列表
good_results = []
for p_temp in ["对位选择性","间位选择性","邻位选择性", "总产率"]: 
    if p_temp == "总产率":
        y = data["ton"].values 
    else:
        y = data[p_temp].values*data["ton"].values   #y=某个选择性ton
        
    unique_X = np.unique(X, axis = 0)                #x=多肽序列并去重
    average_Y = np.array([y[(X == val).all(axis = 1)].mean() for val in unique_X])    #y平均
    ###ton从高到低的新列表
    idx = np.argsort(average_Y)[::-1]                #ton从大到小排
    unique_X = unique_X[idx]                         #覆盖之前没排大小的x
    average_Y = average_Y[idx]                       #覆盖之前没排大小的y

    # select top 10 results
    good_results.append(
        unique_X[0:10]
    )                                                #取前10

In [4]:
performed_exp = [list(i) for i in X]   #遍历数据中的每个序列，并将每个序列转换为一个列表，然后将这些列表组成一个新的列表赋值给 performed_exp
candidates = []                        #存储生成的变异候选序列。每当生成一个新的候选序列，它就会被添加到这个列表中
candidate_elite_type = []              #存储突变前的好结果序列
output_data = []
# define the pool for amino acid
sampling_pool = list('ACDEFGHILMNPQSTVYRWK')                              #氨基酸池
while len(candidates) < 10:                                               #启动一个 while 循环，确保生成的候选序列数量达到20个
    elite_type_index = np.random.randint(len(good_results))               #随机选择一条好数据
    elite_index = np.random.randint(len(good_results[elite_type_index]))  #生成一个变异后的新序列
    elite_to_add = list(mutation(good_results[elite_type_index][elite_index], possible_pool = sampling_pool))  #序列添加到列表里
    if (elite_to_add not in candidates) and (elite_to_add not in performed_exp):
        print(elite_type_index)
        print(20*"="+"Before"+20*"=")
        print(good_results[elite_type_index][elite_index])
        print(20*"="+"After"+20*"=")
        print(elite_to_add)
        candidates.append((elite_to_add))                                 #将新的变异序列 elite_to_add 添加到 candidates 列表中
        candidate_elite_type.append(elite_type_index)                     #将当前序列的精英类型索引 elite_type_index 添加到 candidate_elite_type 列表中
        output_data.append([elite_type_index, ''.join(elite_to_add)])     #将当前精英类型索引和新的变异序列（通过 ''.join 将列表转换为字符串）作为一对元组添加到 output_data 列表中
        output_array = np.array(output_data, dtype=str)                   #将 output_data 列表转换为 NumPy 数组，数据类型为字符串

2
====================Before====================
['G' 'V' 'M' 'S' 'K' 'G' 'T' 'F' 'L' 'Q']
====================After====================
['G', 'V', 'M', 'W', 'K', 'G', 'T', 'D', 'C', 'Q']
1
====================Before====================
['A' 'Q' 'M' 'T' 'K' 'S' 'T' 'H' 'M' 'P']
====================After====================
['A', 'Q', 'M', 'T', 'K', 'S', 'V', 'H', 'W', 'P']
3
====================Before====================
['F' 'Q' 'S' 'T' 'K' 'S' 'Y' 'H' 'V' 'I']
====================After====================
['W', 'Q', 'S', 'S', 'K', 'S', 'T', 'H', 'V', 'I']
2
====================Before====================
['P' 'R' 'M' 'I' 'K' 'G' 'T' 'F' 'M' 'L']
====================After====================
['P', 'R', 'M', 'I', 'K', 'R', 'T', 'F', 'M', 'L']
0
====================Before====================
['A' 'Q' 'M' 'T' 'K' 'S' 'T' 'H' 'M' 'P']
====================After====================
['L', 'Q', 'N', 'I', 'K', 'S', 'H', 'H', 'M', 'P']
3
====================Before====================
['M' 'P' 'I

In [5]:
np.savetxt("peptide_0_241014.csv", np.array(output_array), fmt='%s',delimiter=',')